# Retake exam: 20 June 2023

## Topic of the exam: Understand top players of global land use and CO2 emissions 

Please ensure that you run each cell (ctrl+enter) so that your inputs are saved 

All questions have either coding cell or a text cell, or a combination of the two. 

You are allowed to create additional cells for each answer to facilitate your work  

## Data preparation: Download and import exiobase

Data to be used in the exam: **year 2019 product by product format**

Population data to be downloaded together with the script

Link to the data: https://zenodo.org/record/5589597

In [1]:
# Import modules
import pandas as pd
import numpy as np

In [2]:
# Import here your data
path = "../data/IOT_2019_pxp/"
A = pd.read_csv(f'{path}A.txt', sep='\t', index_col=[0, 1], header=[0, 1])
Y = pd.read_csv(f'{path}Y.txt' , sep='\t', index_col=[0, 1], header=[0, 1])

In [3]:
# Import satellite accounts
F_sat = pd.read_csv(f'{path}satellite/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_sat_hh = pd.read_csv(f'{path}satellite/F_y.txt' , sep='\t', index_col=[0], header=[0, 1])

In [4]:
# Import impact accounts
F_imp = pd.read_csv(f'{path}impacts/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_imp_hh = pd.read_csv(f'{path}impacts/F_y.txt' , sep='\t', index_col=[0], header=[0, 1])

In [5]:
pp_data = pd.read_excel('../data/exiobase_PopulationGDP_1995_2019.xlsx',sheet_name='Population', index_col=[0, 1, 2])

pp_2019 = pp_data.y2019

## Question 1: Territorial accounting (10 points)

### Q1.1 Can you identify the top 3 countries with the highest per capita domestic land use in 2019? (10 points) 

Hint 1: use the environmental impact with keywords: "Land use Crop, Forest, Pasture"

Hint 2: when loc returns a DataFrame instead of a Series you can use .squeeze() to obtain the Series

In [6]:
first_indicator = "Land use Crop, Forest, Pasture"

In [7]:
F_1_ = F_imp.loc[F_imp.index.str.contains(first_indicator),:].squeeze()
F_1_hh = F_imp_hh.loc[F_imp_hh.index.str.contains(first_indicator),:].squeeze()
F_1_tot_ter = F_1_.groupby(axis=0, level=0, sort=False).sum() + F_1_hh.groupby(axis=0, level=0, sort=False).sum()

In [8]:
F_1_pp = F_1_tot_ter/pp_2019.values
F_1_pp.sort_values(ascending=False).head(3)

region
AU    0.095712
CA    0.059431
LV    0.045612
Name: Land use Crop, Forest, Pasture, dtype: float64

## Question 2: Footprint analysis (10 points)

### Q2.1 Which 3 regions had the highest per capita footprint of land use in 2019? (10 points)

In [9]:
# Calculate the rest of MRIO variables
I = np.identity(A.shape[0])
L = np.linalg.inv(I-A)
x = L @ Y.sum(axis=1)

In [10]:
x_ = x.copy()
x_[x_!=0] = 1/x_[x_!=0]
inv_diag_x_ = np.diag(x_)

In [11]:
# Intensities of land use
f_1 = F_1_ @ inv_diag_x_ 

In [12]:
Y_reg = Y.groupby(level=0, axis=1, sort=False).sum()

In [13]:
e_1 = f_1 @ L @ Y_reg + F_1_hh.groupby(level=0, axis=0, sort=False).sum()

In [14]:
e_1_pp = e_1/pp_2019.values
e_1_pp.sort_values(ascending=False).head(3)

region
LU    0.062014
FI    0.035844
AU    0.032664
dtype: float64

## Questions 3: Trade (15 points)

### Q3.1: Were the regions found in Q1 and Q2 net importers or net exporter of per capita land use in 2019? (3 points)

In [15]:
identified_regions = ["AU", "CA", "LV", "LU", "FI", "AU"]

In [16]:
# Insert here your calculations
net_import = e_1_pp - F_1_pp.values
net_import[identified_regions].sort_values(ascending=False)

region
LU    0.056432
FI   -0.004770
CA   -0.028109
LV   -0.028546
AU   -0.063048
AU   -0.063048
dtype: float64

All selected regions were net exporters per capita of land use except for Luxemburg which is a net importer

### Q3.2: Taking into account all exiobase regions, perform MRIO calculations and identify the top three regions that were net per capita importers of land use in 2019 (12 points)

In [17]:
E_1 = np.diag(f_1) @ L @ Y_reg

E_1.index = Y.index

region_labels = Y.index.to_frame(index=False).region.unique()
sector_labels = Y.index.to_frame(index=False).sector.unique()

In [18]:
CBA_1 = E_1.sum(axis=0)
PBA_1 = E_1.groupby(level=0, axis=0, sort=False).sum().sum(axis=1)

In [19]:
trade_rel_1_pp = CBA_1/pp_2019.values - PBA_1/pp_2019.values

trade_rel_1_pp_top3 = trade_rel_1_pp.sort_values(ascending=False).head(3)

trade_rel_1_pp_top3

region
LU    0.056432
BE    0.016689
TW    0.016242
dtype: float64

## Question 4: Dashboard analysis - Land Use vs. CO2 emissions (25 points)

### Q4.1 Identify global regions with the highest total land use and compare them to their CO2 emissions per capita and Land use per capita. (10 points)

Hint 1: Replicate Qs 2-3 for CO2 emissions

Hint 2: use the environmental extensions with keywords: "CO2 - combustion - air"

In [20]:
e_1.sort_values(ascending=False).head(3)

region
CN    1.501028e+07
WF    1.456155e+07
US    8.501668e+06
dtype: float64

In [21]:
second_indicator = "CO2 - combustion - air"

In [22]:
F_2 = F_sat.loc[F_sat.index.str.contains(second_indicator)].squeeze(axis=0)
F_2_hh = F_sat_hh.loc[F_sat_hh.index.str.contains(second_indicator)].squeeze(axis=0)

# Total territorial emissions
F_2_tot_ter = F_2.groupby(level=0, axis=0, sort=False).sum() + F_2_hh.groupby(level=0, axis=0, sort=False).sum()

In [23]:
# Intensities of CO2 emissions
f_2 = F_2 @ inv_diag_x_ 

In [24]:
E_2 = np.diag(f_2) @ L @ Y_reg
E_2.index = Y.index

In [25]:
CBA_2 = E_2.sum(axis=0)
PBA_2 = E_2.groupby(level=0, axis=0, sort=False).sum().sum(axis=1)

In [26]:
trade_rel_2_pp = CBA_2/pp_2019.values - PBA_2/pp_2019.values

focus_regions_trade_pp_2 = trade_rel_2_pp.loc[["CN","WF", "US"]]
focus_regions_trade_pp_2

region
CN    -648.196123
WF     197.176026
US    2179.502166
dtype: float64

In [27]:
focus_regions_trade_pp_1 = trade_rel_1_pp.loc[["CN","WF", "US"]]
focus_regions_trade_pp_1

region
CN    0.003174
WF   -0.001962
US    0.003975
dtype: float64

In [28]:
dashboard = pd.concat([focus_regions_trade_pp_1, focus_regions_trade_pp_2],axis=1)
dashboard.columns = [first_indicator + " - km2/PP", second_indicator  + " - kg CO2-ep/PP"]
dashboard

,"Land use Crop, Forest, Pasture - km2/PP",CO2 - combustion - air - kg CO2-ep/PP
region,,
CN,0.003174,-648.196123
WF,-0.001962,197.176026
US,0.003975,2179.502166


### Q4.2 Focusing on the two indicators, describe three significantly different findings about the main land use players of the world (15 points)

The description for each finding should:

1) be supported by numerical results you obtained from above calculations and based on comparisons of numerical results; 

2) contain your reflections about the land use stress and problems worldwide


_write your answer here_